In [7]:
# Submission File Format
# You should submit a csv file with exactly 418 entries plus a header row. Your submission will show an error if you have extra columns (beyond PassengerId and Survived) or rows.
# 
# The file should have exactly 2 columns:
# PassengerId (sorted in any order)
# Survived (contains your binary predictions: 1 for survived, 0 for deceased)

# Set up Environment

In [8]:
import pandas as pd
import numpy as np

import sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier

# Clean Up Data

In [13]:
train_kaggle = pd.read_csv("train.csv")
test_kaggle = pd.read_csv("test.csv")

# Checking it out
# print(train.head())
# print(train.columns)
# train.info()

# The Cleaning Checklist
# 1) Dropping unneccessary columns
to_drop = ['Name','Ticket','Cabin','Parch', 'SibSp']
train_kaggle.drop(to_drop, inplace=True, axis=1)
test_kaggle.drop(to_drop, inplace=True, axis=1)

# print(train.head())

# 2) Changing the index of the dataframes - is fine
# 3) Using .str method to clean columns
# 4) Using the DataFrame.applymap() function to clean the entire dataset, element-wise
# 5) Renaming columns to a more recognizable set of labels - fine
# 6) Skipping unnecessary rows in a CSV file - fine

# Convert Columns to categories
train_kaggle["Sex"] = train_kaggle["Sex"].astype("category").cat.codes
train_kaggle["Embarked"] = train_kaggle["Embarked"].astype("category").cat.codes

# Func: clean_nan
# Purpose: find NaN and use the average value to fill in
# def clean_nan(column):
# values = {'PassengerId':0,'Pclass':2, 'Sex':0, 
#           'Age':train_kaggle['Age'].mean, 'Fare':train_kaggle['Fare'].mean,
#          'Embarked':1}
# only the age has NaN
average_age = np.round(train_kaggle["Age"].mean())
print(average_age)
train_kaggle.fillna(value= average_age)
    
    

# Print data set 
train_kaggle.info()


30.0


,PassengerId,Survived,Pclass,Sex,Age,Fare,Embarked
0,1,0,3,1,22.0,7.2500,2
1,2,1,1,0,38.0,71.2833,0
2,3,1,3,0,26.0,7.9250,2
3,4,1,1,0,35.0,53.1000,2
4,5,0,3,1,35.0,8.0500,2
5,6,0,3,1,30.0,8.4583,1
6,7,0,1,1,54.0,51.8625,2
7,8,0,3,1,2.0,21.0750,2
8,9,1,3,0,27.0,11.1333,2
9,10,1,2,0,14.0,30.0708,0


# What are we using as our indicator?

In [14]:
# Split training set into train & test
train, test = train_test_split(train_kaggle, test_size = 0.2)

# train sets
train_x = train.drop('Survived', 1)
train_y = train['Survived']

# test sets
test_x = test.drop('Survived', 1)
test_y = test['Survived']

# Model/Train/Predict

In [15]:
# model
tree = DecisionTreeClassifier(criterion = "entropy")

# train
tree.fit(train_x, train_y)

# predict
tree_predictions = tree.predict(test_x)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').